#Name: Ankita Ghosh
#Reg number: 180905354
#Sec:A Roll number:41

Colab Link: https://colab.research.google.com/drive/1TiW9hfEKNGQihyuWlQmin9fd-_0cBurD?usp=sharing

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

##26th May: 1D convolution 
#####Write a program in CUDA which performs convolution operation on one dimensional input array N of size width using a mask array M of size mask_width to produce the resultant one dimensional array P of size width. Find the time taken by the kernel.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include"cuda_runtime.h"
#include"device_launch_parameters.h"

__global__ void conv1D(int *N,int *M,int *P,int Mask_width,int Width)
{
  int i=blockIdx.x*blockDim.x+threadIdx.x;
  int p=0;
  int NStartPoint=i-(Mask_width/2); 
  for(int j=0;j<Mask_width;j++)
  {
    if(NStartPoint+j>=0 && NStartPoint+j<Width)
    {
      p+=N[NStartPoint+j]*M[j];
    }
  }
  P[i]=p;
}

int main()
{
    int Width=10;
    int Mask_width=3;

    int N[]={0,1,2,3,4,5,6,7,8,9};
    printf("\n\nInput array:\n");
    for(int i=0;i<Width;i++)
    {
        printf("%d  ",N[i]);
    }
    int M[]={1,2,1};
    printf("\n\nMask array:\n");
    for(int i=0;i<Mask_width;i++)
    {
        printf("%d  ",M[i]);
    }
    
    int P[Width];
    int size=Width*sizeof(int);
    int *d_N,*d_M,*d_P;
    cudaEvent_t begin,end;

    cudaMalloc((void**)&d_N,size);
	  cudaMalloc((void**)&d_M,Mask_width*sizeof(int));
	  cudaMalloc((void**)&d_P,size);

	  cudaMemcpy(d_N,N,size,cudaMemcpyHostToDevice);
	  cudaMemcpy(d_M,M,Mask_width*sizeof(int),cudaMemcpyHostToDevice);
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventRecord(begin,0);

    conv1D<<<1,Width>>>(d_N,d_M,d_P,Mask_width,Width);
    cudaEventRecord(end,0);
    cudaEventSynchronize(end);
    float timeTaken;
    cudaEventElapsedTime(&timeTaken,begin,end);

    cudaMemcpy(P,d_P,size,cudaMemcpyDeviceToHost);
    printf("\n\nOutput array:\n");
    for(int i=0;i<Width;i++)
    {
        printf("%d  ",P[i]);
    }
    printf("\n\nTime taken by the kernel:\n%f",timeTaken);

    cudaFree(d_N);
	  cudaFree(d_M);
	  cudaFree(d_P);
    return 0;
}



Input array:
0  1  2  3  4  5  6  7  8  9  

Mask array:
1  2  1  

Output array:
1  4  8  12  16  20  24  28  32  26  

Time taken by the kernel:
0.019456


#Question 1
#####Write a CUDA program to perform convolution operation on one dimensional input array N of size width using a mask array M of size mask_width to produce the resultant one dimensional array P of size width using constant Memory for Mask array.  Add another kernel function to the same program to perform 1D convolution using shared memory. Find and display the time taken by both the kernels.

In [ ]:
 %%cu
#include<stdio.h>
#include<stdlib.h>
#include"cuda_runtime.h"
#include"device_launch_parameters.h"
#define MAX_MASK_WIDTH 10

__constant__ int M[MAX_MASK_WIDTH];

__global__ void conv1Dcm(int *N,int *P,int Mask_width,int Width)
{
  int i=blockIdx.x*blockDim.x+threadIdx.x;
  int p=0;
  int NStartPoint=i-(Mask_width/2); 
  for(int j=0;j<Mask_width;j++)
  {
    if(NStartPoint+j>=0 && NStartPoint+j<Width)
    {
      p+=N[NStartPoint+j]*M[j];
    }
  }
  P[i]=p;
}

__global__ void conv1Dsm(int *N,int *P,int Mask_width,int Width)
{
  extern __shared__ int temp_array[];
  int id=blockIdx.x*blockDim.x+threadIdx.x;
  temp_array[threadIdx.x]=N[id];
  __syncthreads();
  int p=0;
  for(int i=0;i<Mask_width;i++)
  {
    if(threadIdx.x+i>=blockDim.x)
    {
      p+=N[id+i]*M[i];
    }
    else
    {
      p+=temp_array[threadIdx.x+i]*M[i];
    }
  }
  P[id]=p;
}

int main()
{
    int Width=10;
    int Mask_width=3;

    int h_N[]={0,1,2,3,4,5,6,7,8,9};
    printf("\n\nInput array:\n");
    for(int i=0;i<Width;i++)
    {
        printf("%d  ",h_N[i]);
    }
    int h_M[]={1,2,1};
    printf("\n\nMask array:\n");
    for(int i=0;i<Mask_width;i++)
    {
        printf("%d ",h_M[i]);
    }
 
    int h_P[Width];
    int size=Width*sizeof(int);
    int *d_N,*d_P;
    float eTime1,eTime2;
    cudaEvent_t begin,end,begin1,end1;

    cudaMalloc((void**)&d_N,size);
	  cudaMalloc((void**)&d_P,size);

	  cudaMemcpy(d_N,h_N,size,cudaMemcpyHostToDevice);
	  cudaMemcpyToSymbol(M,h_M,Mask_width*sizeof(int));
    cudaEventCreate(&begin);
    cudaEventCreate(&end);
    cudaEventCreate(&begin1);
    cudaEventCreate(&end1);
    cudaEventRecord(begin,0);

    conv1Dcm<<<1,Width>>>(d_N,d_P,Mask_width,Width);
    cudaEventRecord(end,0);
    cudaEventSynchronize(end);
    cudaEventElapsedTime(&eTime1,begin,end);

    cudaMemcpy(h_P,d_P,size,cudaMemcpyDeviceToHost);
    printf("\n\nOutput array from kernel 1:\n");
    for(int i=0;i<Width;i++)
    {
        printf("%d   ",h_P[i]);
    }
    printf("\nTime taken by kernel 1: %f\n",eTime1);
    cudaEventRecord(begin1,0);

    conv1Dsm<<<1,Width>>>(d_N,d_P,Mask_width,Width);
    cudaEventRecord(end1,0);
    cudaEventSynchronize(end1);
    cudaEventElapsedTime(&eTime2,begin1,end1);

    cudaMemcpy(h_P,d_P,size,cudaMemcpyDeviceToHost);
    printf("\n\nOutput array from kernel 2:\n");
    for(int i=0;i<Width;i++)
    {
        printf("%d   ",h_P[i]);
    }
    printf("\nTime taken by kernel 2: %f",eTime2);

    cudaFree(d_N);
	  cudaFree(d_P);
    return 0;
}



Input array:
0  1  2  3  4  5  6  7  8  9  

Mask array:
1 2 1 

Output array from kernel 1:
1   4   8   12   16   20   24   28   32   26   
Time taken by kernel 1: 0.021504


Output array from kernel 2:
1   4   8   12   16   20   24   28   32   26   
Time taken by kernel 2: 0.000000


#Question 2
#####Write a program in CUDA to perform parallel Sparse Matrix - Vector Multiplication using compressed sparse row (CSR) storage format. Represent the input sparse matrix in CSR format in the host code.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include"cuda_runtime.h"
#include"device_launch_parameters.h"

__global__ void Sparse_CSR(int num_rows,int *data,int *col_index,int *row_ptr,int *x,int *y)
{
  int row=threadIdx.x;
  if(row<num_rows)
  {
    int dot=0;
    int row_start=row_ptr[row];
    int row_end=row_ptr[row+1];
    for(int i=row_start;i<row_end;i++)
    {
      dot+= data[i]*x[col_index[i]];
    }
    y[row]=dot;
  }
}

int main()
{

    int n=3;
    int y[n],row_ptr[n+1];
    int ipmat[n][n]={{1,2,3},{4,5,6},{7,8,9}};
    int x[]={1,2,3};
    int nonzerocount=0;

    printf("\nInput Array:\n");
    for(int i=0;i<n;i++)
    {
      row_ptr[i]=nonzerocount;
      for(int j=0;j<n;j++)
      {
        if(ipmat[i][j]!=0)
        {
          nonzerocount++;
        }
        printf("%d\t",ipmat[i][j]);
      }
      printf("\n");
    }
    row_ptr[n]=nonzerocount;
    int data[nonzerocount],col_index[nonzerocount];
    int k=0;
    //finding data and col_index array
    for(int i=0;i<n;i++)
    {
      for(int j=0;j<n;j++)
      {
        if(ipmat[i][j]!=0)
        {
          data[k]=ipmat[i][j];
          col_index[k++]=j;
        }
      }
    }
    printf("\n\nValue Array:\n");
    for(int i=0;i<nonzerocount;i++)
    {
      printf("%d  ",data[i]);
    }
    printf("\n\nColumn Index Array:\n");
    for(int i=0;i<nonzerocount;i++)
    {
      printf("%d  ",col_index[i]);
    }
    printf("\n\nRow Pointer Array:\n");
    for(int i=0;i<=n;i++)
    {
      printf("%d  ",row_ptr[i]);
    }
    printf("\n\nVector:\n");
    for(int i=0;i<n;i++)
    {
      printf("%d  ",x[i]);
    }
    int *d_data,*d_col_index,*d_row_ptr,*d_x,*d_y;

    cudaMalloc((void**)&d_data,nonzerocount*sizeof(int));
    cudaMalloc((void**)&d_col_index,nonzerocount*sizeof(int));
    cudaMalloc((void**)&d_row_ptr,(n+1)*sizeof(int));
	  cudaMalloc((void**)&d_x,n*sizeof(int));
	  cudaMalloc((void**)&d_y,n*sizeof(int));

    cudaMemcpy(d_data,data,nonzerocount*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_col_index,col_index,nonzerocount*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_row_ptr,row_ptr,(n+1)*sizeof(int),cudaMemcpyHostToDevice);
	  cudaMemcpy(d_x,x,n*sizeof(int),cudaMemcpyHostToDevice);

    Sparse_CSR<<<1,n>>>(n,d_data,d_col_index,d_row_ptr,d_x,d_y);

    cudaMemcpy(y,d_y,n*sizeof(int),cudaMemcpyDeviceToHost);

    printf("\n\nResult:\n");
    for(int i=0;i<n;i++)
    {
        printf("%d  ",y[i]);
    }

    cudaFree(d_data);
    cudaFree(d_col_index);
    cudaFree(d_row_ptr);
	  cudaFree(d_x);
	  cudaFree(d_y);
    return 0;
}


Input Array:
1	2	3	
4	5	6	
7	8	9	


Value Array:
1  2  3  4  5  6  7  8  9  

Column Index Array:
0  1  2  0  1  2  0  1  2  

Row Pointer Array:
0  3  6  9  

Vector:
1  2  3  

Result:
14  32  50  


#Question 3
#####Write a program in CUDA to perform matrix multiplication using 2D Grid and 2D Block.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ void mulmat(const int *a, const int *b, int *c, int m,int n,int o)
{
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  c[row * o + col] = 0;

  for (int k = 0; k < n; k++) {
    c[row * o + col] += a[row * n + k] * b[k * o + col];
  }
}

int main()
{
  int size = sizeof(int);
  int m=4,n=2,o=4;
  int a[m][n]={{1,2},{3,4},{5,6},{7,8}};
  int b[n][o]={{1,2,3,4},{1,2,3,4}};
  int c[m][o];
 
  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a,m*n*size);
  cudaMalloc(&d_b,n*o*size);
  cudaMalloc(&d_c,m*o*size);

  cudaMemcpy(d_a,a,m*n*size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,b,n*o*size, cudaMemcpyHostToDevice);

  int thread=2;
  dim3 threads(thread,thread);
  dim3 blocks((m*o)/(4*thread),(m*o)/(4*thread));

  mulmat<<<blocks, threads>>>(d_a, d_b, d_c,m,n,o);

  cudaMemcpy(c, d_c,m*o*size, cudaMemcpyDeviceToHost);
 
  printf("\nMatrix 1:\n");
  for(int i=0;i<m;i++)
  {
    for(int j=0;j<n;j++)
    {
      printf("%d\t",a[i][j]);
    }
    printf("\n");
  }
 
  printf("\nMatrix 2:\n");
  for(int i=0;i<n;i++)
  {
    for(int j=0;j<o;j++)
    {
      printf("%d\t",b[i][j]);
    }
    printf("\n");
  }
 
  printf("\nFinal Matrix: \n");
  for(int i=0;i<m;i++)
  {
    for(int j=0;j<o;j++)
    {
      printf("%d\t",c[i][j]);
    }
    printf("\n");
  }

  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}


Matrix 1:
1	2	
3	4	
5	6	
7	8	

Matrix 2:
1	2	3	4	
1	2	3	4	

Final Matrix: 
3	6	9	12	
7	14	21	28	
11	22	33	44	
15	30	45	60	

